# Nye kanon for lærerutdanningen - februar 2020

Nedenfor går vi gjennom en notebook for å studere et mindre korpus av bøker (under 100), finne ut noe av det tematiske, hvordan ord henger sammen gjennom korpuset, samt bruk av metadata og lesbarhetsindeks (TTR).

Først henter vi inn det som trengs for at notebooken skal virke.

In [20]:
import dhlab.nbtext as nb
from dhlab.module_update import css, code_toggle
import pandas as pd
import qgrid as qg
css()

### Her defineres noen hjelpekommandoer for kanon-prosjektet

Det er ikke nødvendig å forstå definisjonen av kommandoene, men greit å vite hva de gjør. Hopp over i første omgang, og sjekk etter å ha sett hvordan kommandoen brukes.

#### hjelpekommando for å sjekke ord


In [21]:
sjekk_ord = lambda x: texts.loc[x].dropna()

#### en kommando som viser en dataramme med tall som et varmekart - fin å bruke for dokument-term-matriser.


In [22]:
varmekart = lambda t, top=5: t.head(top).fillna(0).style.background_gradient()

#### Kommando for å se på ord gjennom hele korpuset

Variabelen `dtm` er dokument-term-matrisen, og `ordliste` er en liste med ord skilt med blanke tegn.

In [23]:
def oversikt(dtm, ordliste):
    if isinstance(ordliste, str):
        ordliste = ordliste.split() 
    df = dtm.reindex(ordliste).loc[ordliste].transpose().fillna(0)
    df.index = bok_info(nb.metadata(list(df.index)))
    return df.style.background_gradient()

#### bok_info smelter sammen informasjon om metadata



In [24]:
bok_info = lambda b: ['-'.join([str(y) for y in x[0:4]]) for x in b]

#### kommandoen `bøker` tar et `søkeord` og viser en liste over de bøker ordet forekommer i, sortert på frekvens.



In [25]:
def bøker(søkeord):
    try:
        data = sjekk_ord(søkeord)
        df = nb.frame(nb.metadata(list(data.index)), "urn forfatter år tittel subtittel forlag sjanger".split())["urn forfatter år tittel subtittel forlag sjanger".split()]
        df[søkeord] = data.values
        df = df.sort_values(by=søkeord, ascending = False)
    except:
        df = pd.DataFrame()
    return df

marc21

## Lag et korpus

Korpus bygges gjennom å velge bøker fra bokhylla. Alle bøker har en såkalt URN (Universal Resource Name) som entydig peker på den digitale kopien. Den består av en serie bilder av boksidene, i tillegg til den OCR-behandlede teksten. Det er teksten som gjøres gjenstand for analyse.

Et korpus velges gjennom kommandoen `book_corpus`, som tar et sett parametre for å begrense utvalget, som periode, dewey, emneord, forfatter, tittel og språk, med mer.

In [44]:
korpus = nb.book_corpus(subject = 'helse', period = (1980, 2000), lang='nob', limit = 50 )

In [45]:
korpus

,urn,author,title,year
0,2014052006013,"Bø, Kari",Gymnastikk i tiden,1989
1,2009020204153,"Trauer, Tom",Hvordan bearbeide stress,1988
2,2013062106020,,Miljøkunnskap,1985
3,2009121700053,,Samdata psykiatri,1998
4,2009102000054,"Dahl, Hans A.",Kroppen,1994
5,2011012609060,,Forskning om kvinners helse,1996
6,2010030204049,"Lippestad, Jan",Evaluering av habiliteringstjenestene (spesial...,1999
7,2010050705083,"Gogstad, Anders Chr.",Helse og hakekors,1991
8,2007110200001,"Hodgson, John",Bioteknologi,1991
9,2011041408115,"Daatland, Svein Olav",Aldring og samfunn,2000


In [47]:
nb.frame(nb.best_book_urn(word='demokrati', period=(1880, 1900))).head()

,0,1,2,3
0,2008040200084,"Ullmann, V.",Haandbog i verdenshistorien,1897
1,2006082800101,"Aars, J.",Sokrates,1882
2,2015021108139,"O'Rell, Max",Jonathan og hans fastland,1889
3,2008012502003,"Bjørnson, Bjørnstjerne",Mine Brev til Petersburgskija Vjedomosti m.m,1898
4,2006112200011,"Troye, Vilhelm",Thomas Carlyle,1889


In [48]:
nb.frame(nb.best_book_urn(word='demokrati', period=(1900, 1950))).head()

,0,1,2,3
0,2007020600009,,Nordisk demokrati,1949
1,2010120705064,"Mogens, Victor",Politikk og fornuft,1950
2,2007020201094,,Verdenshistorie,1939
3,2008041600062,"Hesstvedt, Knut","Norge ved skilleveien, en kultur-cyklisk studi...",1945
4,2011071506015,"Elster, Torolf",Frihet og demokrati,1947


# Behandling

Korpuset kan analyseres med hensyn til tema og lesbarhet, i tillegg til annen statistikk. Det kan også søkes i, for å se litt om kontekster og hva som behandles. Konkordansene kan også gi et innblikk i hva som tematisk behandles.

In [28]:
nb.urn_concordance(word='forelsket', urns = nb.pure_urn(korpus), before = 10, after = 10)

"Gjest hos uvirkeligheten, Granath, Gunilla, 1998",- Det virker som om Jenny og Saskia allerede er,forelsket,"i Elias , sier Matilda og ler ."
"Gjest hos uvirkeligheten, Granath, Gunilla, 1998",at Jakob fikk lese min fabel om hva man blir,forelsket,i - sjelen eller skjønnheten ? Jeg har en følelse
"Gjest hos uvirkeligheten, Granath, Gunilla, 1998",ydmyke av en av guttene i ttende som hun er,forelsket,i .
"Gjest hos uvirkeligheten, Granath, Gunilla, 1998",ikke det er noen i klassen som har vært så,forelsket,som jeg . Ikke engang Charlotta . Jeg har vært
"Gjest hos uvirkeligheten, Granath, Gunilla, 1998",å bli oppspilte . Jack er oppriktig,forelsket,"i Amanda ; de småbryter, og han later som om"
"Barnet og de andre, Bø, Inge, 1995",basis i den greske sagnfiguren Narcissos som ble så tragisk,forelsket,"i sitt eget speilbilde , mener Lasch vi lever i"
"[800 33 866 - SUSS-telefonen], , 1995",Jeg og ei venninne er,forelsket,"i samme gutt , og nå har jeg funnet ut"
"[800 33 866 - SUSS-telefonen], , 1995",", og nå har jeg funnet ut at han er",forelsket,i meg . Hva skal jeg gjøre ?
"[800 33 866 - SUSS-telefonen], , 1995",Så hyggelig at du er,forelsket,og så trist at du har kommet i en slik
"[800 33 866 - SUSS-telefonen], , 1995",om jeg vil være sammen med dem . Jeg er,forelsket,"i den ene , men den andre spurte meg først"
"[800 33 866 - SUSS-telefonen], , 1995",er dumt å bli sammen med noen du ikke er,forelsket,i . Det vil alltid være situasjoner i livet man


# Hent tekstene som frekvenslister over ord

Tekstene kan lastes ned som en såkalt dokument-term-matrise. Den gjør det mulig å se på hvordan ord fordeler gjennom korpuset.

In [29]:
texts = nb.get_corpus_text(nb.pure_urn(korpus), cutoff = 1)

### Slik ser tekstene ut som en dokument-term-matrise

Her viser vi tekstene med et varmekart som gir et raskt overblikk over forskjeller mellom dem.

In [30]:
varmekart(texts)

,2013081606105,2010122006024,2008022500074,2007110904050,2015041508102,2008080804068,2010090208067,2008011504061,2012113008048,2007112704076,2015010648018,2011012008020,2009011400083,2014033108002,2010081022013,2012062708234,2008010400021,2011011208207,2014061105138,2009120104196,2008091700025,2007071800083,2011090208006,2013121106086,2009022600028,2010082008077,2012101208122,2009040300032,2007082001026,2012080208200,2010031603066,2009101300056,2009022304156,2012121306016,2007112701005,2013041106021,2008090904062,2010081608048,2012080108066,2008080100093,2009101304049,2015051308327,2007112804071,2007101504039,2013070808002,2010021500059,2010102008085,2014072308050,2008060904090,2009082501012
.,1102,4055,5364,1459,1244,823,538,1460,536,4622,435,1760,4788,1696,2812,265,4524,1157,621,914,548,2073,1380,548,1830,1941,3135,1170,803,890,4212,595,3828,651,6199,1412,1007,536,472,975,661,1158,951,846,3605,4416,40,3378,1186,2008
",",803,2208,6199,1059,970,727,244,898,177,3308,262,1323,4302,820,3018,87,3374,516,490,518,328,2011,997,356,1921,1466,1432,863,732,668,2295,300,3403,399,7282,527,811,466,422,701,265,578,558,480,1955,2855,26,4034,341,1295
og,635,1921,4501,909,1082,382,281,718,165,3188,184,1499,3666,678,1507,107,3025,692,498,879,368,1391,830,280,1945,2540,1952,835,629,482,1107,255,1910,29,4078,781,818,240,317,618,303,619,551,475,1695,3805,18,7,519,1807
i,594,1027,2923,767,673,277,305,838,272,2154,150,1128,2413,605,945,88,2830,522,275,513,342,952,566,484,874,1659,1857,533,406,372,847,270,1563,114,2442,679,765,558,227,526,619,479,618,626,1189,2593,24,1792,636,1720
er,415,1821,1945,598,453,195,216,676,239,1851,268,500,2249,434,523,105,2014,527,302,397,116,674,679,74,929,846,1574,396,451,306,1116,233,1152,52,2413,427,469,150,181,390,215,445,229,311,352,1247,31,7,498,833


## Spredning av ord

Forekomster av et vilkårlig sett med ord kan visualiseres gjennom korpuset. Varmekartet gir et godt blikk på hvordan ordene fordeler seg og samvarierer.

In [31]:
oversikt(texts, ['Pakistan', 'Sverige', 'India', 'Iran'])

,Pakistan,Sverige,India,Iran
2013081606105--1992-Samordning av lokale kriminalitetsforebyggende tiltak (SLT),0,3,0,0
"2010122006024-Granath, Gunilla-1998-Gjest hos uvirkeligheten",0,8,0,0
"2008022500074-Bø, Inge-1995-Barnet og de andre",0,5,0,0
"2007110904050-Olweus, Dan-1992-Mobbing i skolen",0,7,0,0
"2015041508102-Ommundsen, Yngvar-1989-Forebyggende helsepedagogiske tiltak for unge med utgangspunkt i idrettsmiljø",0,0,0,0
"2008080804068-Røysamb, Espen-1998-Rusbruk og foreldrerelasjoner gjennom ungdomsårene",0,4,0,0
"2010090208067-Moe, Dagfinn-1998-Bruk av belønning for å redusere trafikkulykker blant unge bilførere",0,0,0,0
"2008011504061-Øia, Tormod-1993-Mellom moské og McDonald's",2,3,2,0
"2012113008048-Bugge, Liv Susanne-1988-Bære je' ser asfalt -",0,0,0,0
"2007112704076-Bunkholdt, Vigdis-1993-Praktisk barnevernsarbeid",0,3,0,0


# Metadata

Over lagde vi en hjelpekommando, kalt `bøker`, som tar et ord som parameter, og lister ut alle bøkene som har ordet, sortert på frekvensen. Den gir litt mer detalj om metadata enn kommandoen `oversikt`.

In [32]:
bøker('miljø')

,urn,forfatter,år,tittel,subtittel,forlag,sjanger,miljø
2,2008022500074,"Bø, Inge",1995,Barnet og de andre,nettverk som pedagogisk og sosial ressurs,TANO,notfiction,114.0
7,2011012008020,,1995,Barn og unge - dagens kanarifugler?,sluttrapport,Fylkeskommunen,unknown,55.0
6,2007112704076,"Bunkholdt, Vigdis",1993,Praktisk barnevernsarbeid,,Universitetsforlaget,notfiction,40.0
9,2008010400021,,1993,Ungdom i lokalmiljø,"perspektiv frå pedagogikk, sosiologi, antropol...",Samlaget,notfiction,34.0
10,2011011208207,"Kvalvaag, Heidi M.",1996,"Forebyggende arbeid overfor ""bekymringsungdom""",en evaluering av V13 i Bergen,"Diakonhjemmets høgskolesenter, Forskningsavdel...",unknown,31.0
22,2007112701005,"Tetzchner, Stephen von",1992,Habilitering,tverrfaglig arbeid for mennesker med medfødte ...,Ad Notam Gyldendal,notfiction,26.0
21,2009022304156,"Maier, Henry W.",1997,Barn og unge utenfor familien,utviklingsmuligheter,Ad notam Gyldendal,notfiction,17.0
14,2009022600028,"Theophilakis, Mary",1980,vanskelige skylden,"om selvbilde, skyld og ansvar i behandling",Universitetsforlaget,notfiction,9.0
13,2007071800083,"Jonsson, Letty",1984,Mennesker med psykisk utviklingshemning,,NKS-forl.,notfiction,9.0
24,2010021500059,"Loland, Sigmund",1998,"Idrett, kultur og samfunn",,Universitetsforlaget,unknown,8.0


In [33]:
bøker('fotball')

,urn,forfatter,år,tittel,subtittel,forlag,sjanger,fotball
3,2009011400083,"Hollekim, Ingjerd",2000,Mestring og glede,fra grunnleggende bevegelser til idrett og dans,Gyldendal undervisning,notfiction,48.0
12,2010021500059,"Loland, Sigmund",1998,"Idrett, kultur og samfunn",,Universitetsforlaget,unknown,42.0
1,2008022500074,"Bø, Inge",1995,Barnet og de andre,nettverk som pedagogisk og sosial ressurs,TANO,notfiction,6.0
5,2008010400021,,1993,Ungdom i lokalmiljø,"perspektiv frå pedagogikk, sosiologi, antropol...",Samlaget,notfiction,5.0
6,2008091700025,,1998,Barn og unge,aktuell statistikk,Statistisk sentralbyrå;Oslo : Statistisk sentr...,unknown,5.0
0,2010122006024,"Granath, Gunilla",1998,Gjest hos uvirkeligheten,en 48-årig sjuendeklassings dagbok,Bedre skole;Norsk lærerlag,notfiction,4.0
8,2010031603066,"Edvall, Lilian",1998,For første gang,forelskelse og sex : for tenåringer om tenåringer,Cappelen,notfiction,4.0
11,2015051308327,"Jakobsen, Arne Martin",2000,Motivasjon og idrett,,Høgskolen i Bodø,unknown,4.0
2,2007112704076,"Bunkholdt, Vigdis",1993,Praktisk barnevernsarbeid,,Universitetsforlaget,notfiction,2.0
4,2010081022013,"Coelho, Paulo",1999,Veronika vil dø,,Ex libris,novel,2.0


# Lesbarhetsindekser

Alle bøkene kan beskrives gjennom en lesbarhetsindeks, standarden er STTR (Standardized Type Toke Ratio). Med standarized menes at indeksen er rimelig rettferdig uavhengig av bokens lengde. Her er tallet et gjennomsnitt av å dele inn hver bok i seksjoner på 5000 ord hver.

### Bygg lesbarhetsindeksen

Indeksen bygges med kommandoen `sttr`.

In [34]:
lesbarhet = nb.frame({b:nb.sttr(b) for b in korpus.urn}, name='indeks')

### Henter ut metada for bøkene for orientering

Det følgende er mest Python-kode, som ikke er essensiell for forståelsen av det som skjer, men kan stå som en dokumentasjon på det som er gjort for å lage sluttproduktet vist fire celler under.

In [35]:
bøker_meta = nb.metadata(list(lesbarhet.index))

In [36]:
lesbarhet['info'] = bok_info(bøker_meta)

### Resultatet av lesbarhetsindeksen

In [37]:
nb.frame_sort(lesbarhet, by='indeks')

,indeks,info
2012062708234,0.597195,"2012062708234-Klyve, Arne-1994-Rus som livssti..."
2009082501012,0.398411,2009082501012--1993-Ungdomspolitikk mot et nyt...
2014072308050,0.396169,"2014072308050-Ohlsson, Lars B.-1994-Ungdom som..."
2015041508102,0.386247,"2015041508102-Ommundsen, Yngvar-1989-Forebygge..."
2013081606105,0.382156,2013081606105--1992-Samordning av lokale krimi...
2008022500074,0.380127,"2008022500074-Bø, Inge-1995-Barnet og de andre"
2009040300032,0.369585,2009040300032--1998-Barn og unge med alvorlige...
2014061105138,0.362539,"2014061105138-Stokke, Erle-2000-lille forfatte..."
2012080108066,0.354400,"2012080108066-Tharaldsen, Jorunn-Elise-1998-Un..."
2013041106021,0.350299,2013041106021--1984-Krise- og avlastningsavdel...
